In [1]:
import os

from langchain.chat_models import init_chat_model
ds_llm = init_chat_model(
    "deepseek/deepseek-chat-v3-0324:free", 
    model_provider="openai",
    base_url=os.getenv("OPENROUTER_BASE_URL"), 
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
)



In [ ]:
from pydantic import BaseModel, Field

class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

In [4]:
tools = [add, multiply]

In [ ]:
query = "3 * 12等于多少?"

output = llm_with_tools.invoke(query)

In [7]:
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML
import json

def display_json_with_highlight(output):
    # pprint.pprint(output.to_json(), indent=1)
    formatted_json = json.dumps(output.to_json(), indent=2)
    formatter = HtmlFormatter(style='dracula', noclasses=True) 
    highlighted_html = highlight(formatted_json, JsonLexer(), formatter)
    pygments_style_defs = formatter.get_style_defs('.highlight')
    custom_css = f"""
    <style>
        /* Pygments Dracula 主题的 CSS 规则 */
        {pygments_style_defs}

        /* 覆盖 Pygments 默认字体，并确保使用 Fira Code */
        .highlight, .highlight pre, .highlight code {{
            font-family: "Fira Code", monospace !important;
            font-size: 2 em !important; /* 调整字体大小 */
            line-height: 1.5 !important; /* 调整行高 */
        }}

        /* 确保整个高亮块的背景色和文本颜色是 Dracula 的，并提供一些内边距 */
        /* .output_area 是 Jupyter 输出区域的类 */
        .output_area pre.highlight {{
            background-color: #282a36 !important; /* Dracula 的背景色 */
            color: #f8f8f2 !important; /* Dracula 的默认文本色 */
            padding: 1em !important;
            border-radius: 4px !important;
            overflow-x: auto !important; /* 防止内容溢出，添加滚动条 */
        }}

        /* 对于代码内部的特定 Pygments 颜色类，如果需要微调，可以在这里添加 */
        /* 例如：
        .highlight .k {{ color: #ff79c6 !important; }} /* Dracula 的关键字颜色 */
        */
    </style>
    """
    return highlighted_html+custom_css

HTML(display_json_with_highlight(output))

In [16]:
query = "what is 11 + 49? What is 3 * 12? "
output = llm_with_tools.invoke(query).tool_calls

In [17]:
import pprint
pprint.pprint(output)

[{'args': {'a': 11, 'b': 49},
  'id': 'call_EdQxQm3kSLWgzjELrOEb1A',
  'name': 'add',
  'type': 'tool_call'}]


In [19]:
gpt_llm = init_chat_model("gpt-4o-mini")
gpt_llm_with_tools = gpt_llm.bind_tools(tools)
output = gpt_llm_with_tools.invoke(query).tool_calls

In [20]:
pprint.pprint(output)

[{'args': {'a': 11, 'b': 49},
  'id': 'call_ubkq5agtJ6hvDDpyi8kHMPhe',
  'name': 'add',
  'type': 'tool_call'},
 {'args': {'a': 3, 'b': 12},
  'id': 'call_l4ahmO1mGCB2HPFvBpnIRFPz',
  'name': 'multiply',
  'type': 'tool_call'}]


In [21]:
from langchain_core.output_parsers import PydanticToolsParser

chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
output = chain.invoke(query)

In [23]:
pprint.pprint(output)

[add(a=11, b=49)]


In [24]:
chain = gpt_llm_with_tools | PydanticToolsParser(tools=[add, multiply])
output = chain.invoke(query)

In [25]:
pprint.pprint(output)

[add(a=11, b=49), multiply(a=3, b=12)]


In [26]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

chain = gpt_llm_with_tools | PydanticToolsParser(tools=[add, multiply])
output = chain.invoke(query)

In [27]:
pprint.pprint(output)

[60, 36]


In [28]:
llm = init_chat_model(
    "deepseek/deepseek-chat-v3-0324:free", 
    model_provider="openai",
    base_url=os.getenv("OPENROUTER_BASE_URL"), 
    openai_api_key=os.getenv("OPENROUTER_API_KEY"),
)

tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)
query = "3 * 12等于多少?"
output = llm_with_tools.invoke(query)

In [29]:
pprint.pprint(output.to_json(), indent=1)

{'id': ['langchain', 'schema', 'messages', 'AIMessage'],
 'kwargs': {'additional_kwargs': {'refusal': None,
                                  'tool_calls': [{'function': {'arguments': '{"a":3,"b":12}',
                                                               'name': 'multiply'},
                                                  'id': 'call_w7viZDsjSC-sBc6zxB_k3g',
                                                  'index': 1,
                                                  'type': 'function'}]},
            'content': '',
            'id': 'run--c0bdc09d-df60-47fa-a0ff-f8fafd0818d6-0',
            'invalid_tool_calls': [],
            'response_metadata': {'finish_reason': 'tool_calls',
                                  'id': 'gen-1750005217-JoRLvSCgxWHp8X49O3Cz',
                                  'logprobs': None,
                                  'model_name': 'deepseek/deepseek-chat-v3-0324:free',
                                  'service_tier': None,
                       